# Project - my version

In [1]:
import sys

my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import load_csv_data 

In [2]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)

In [3]:
input_data.shape

(250000, 30)

In [4]:
for i in range(input_data.shape[1]):
    input_data[np.where(input_data[:,i]==-999),i] = 0


##  R^2 adjusted - choosing the features

In [10]:
# Subdived the X features space in single features
features = []
all_features = ['DER_mass_MMC', 'DER_mass_transverse_met_lep','DER_mass_vis','DER_pt_h', 'DER_deltaeta_jet_jet',
                'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
                'DER_pt_ratio_lep_tau','DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 
                'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
                'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt','PRI_jet_leading_eta', 'PRI_jet_leading_phi',
                'PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi','PRI_jet_all_pt']
for i in range(len(all_features)):
    features.append((i,all_features[i]))
    


In [11]:
# STEP-WISE
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
K = 0 #needed for the R^2 adjusted
w0 = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(yb)

y = X.dot(w0) # our model output
e = yb - y
sse = np.sum(e**2)
sst = np.sum((yb - yb.mean())**2)
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (K/(n-K-1)*(1-R2))

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(yb)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = X.dot(ws)
        e = yb - y
        SSE = np.sum(e**2)
        SST = np.sum((yb- yb.mean())**2)
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        #idx_features.append(np.where(all_candidates[:,ind_max] == input_data))
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1])
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        


-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep
-------------------------------------------------
Feature chosen:  PRI_tau_pt
-------------------------------------------------
Feature chosen:  DER_prodeta_jet_jet
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality
-------------------------------------------------
Feature chosen:  DER_deltar_tau_lep
-------------------------------------------------
Feature chosen:  DER_mass_vis
-------------------------------------------------
Feature chosen:  PRI_lep_pt
-------------------------------------------------
Feature chosen:  DER_pt_ratio_lep_tau
-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality
-------------------------------------------------
Feature chosen:  PRI_met_sumet
-------------------------------------------------
Feature chosen:  PRI_met
-------------------------------------------------
Feature chosen:  DER_m

In [12]:
best_R2adj

[0.12349810213950073,
 0.15812705121789919,
 0.17951656712263689,
 0.19145139756206597,
 0.20056837340415318,
 0.20941610738860805,
 0.21772431095196917,
 0.22603930769897096,
 0.23221052586314267,
 0.23507151106960303,
 0.24089710530398334,
 0.24276279305340798,
 0.24496721328203927,
 0.2467242281031215,
 0.24712956555998059,
 0.24727284460920373,
 0.24733914906148918,
 0.24735847303902625,
 0.24737703922184925,
 0.24738269455494491,
 0.24738361708392093,
 0.24738297499122325]

In [13]:
idx_features

[1, 13, 6, 11, 7, 2, 16, 10, 12, 21, 19, 5, 9, 23, 0, 26, 3, 22, 4, 18, 28]